In [1]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama, OllamaEmbeddings
import pandas as pd
from langchain_community.document_loaders import CSVLoader

In [2]:
CHAT_MODEL_NAME = 'llama3.1:8b'
EMBED_MODEL_NAME = 'nomic-embed-text:v1.5'
FILE_PATH = 'LK_modified.xlsx - Вопрос ответ.csv'

llm = ChatOllama(model=CHAT_MODEL_NAME, temperature=0.1)

In [3]:
df = pd.read_csv(FILE_PATH)
df.tail()

,id,question,content,category
1683,1683,как убрать бота,Чат-бот находится в стадии пилотирования и обу...,поддержка
1684,1684,мне сказали создать заявку в поддержке,Для оформления обращения в техническую поддерж...,оператор
1685,1685,выход,Для оформления обращения в техническую поддерж...,оператор
1686,1686,вернуться на старый портал,Чат-бот находится в стадии пилотирования и обу...,поддержка
1687,1687,нужен оператор,Чат-бот находится в стадии пилотирования и обу...,поддержка


In [4]:
loader = CSVLoader(
    file_path=FILE_PATH,
    metadata_columns=['id', 'category'],
    content_columns=['question', 'content'],
    encoding='utf-8',
)
documents = loader.load()
documents[:5]

[Document(metadata={'source': 'LK_modified.xlsx - Вопрос ответ.csv', 'row': 0, 'id': '0', 'category': 'автомобиль'}, page_content='question: Я сменил автомобить, на учет еще не поставил, могу ли я заправляться по топливной карте?\ncontent: Для внесения данных по личному автомобилю обратитесь, пожалуйста, к своему руководителю для создания заявки по теме "Изменение режима характера работы", подтема "Установка РХР и топливной карты". В комментариях опишите ситуацию и приложите ПТС, СТС, страховой полис и водительское удостоверение.'),
 Document(metadata={'source': 'LK_modified.xlsx - Вопрос ответ.csv', 'row': 1, 'id': '1', 'category': 'автомобиль'}, page_content='question: Не отображается автомобиль в личном кабинете.\ncontent: Для внесения данных по личному автомобилю обратитесь, пожалуйста, к своему руководителю для создания заявки по теме "Изменение режима характера работы", подтема "Установка РХР и топливной карты". В комментариях опишите ситуацию и приложите ПТС, СТС, страховой поли

In [5]:
vectorstore = Chroma.from_documents(
    collection_name='question_answer_collection',
    documents=documents,
    embedding=OllamaEmbeddings(model=EMBED_MODEL_NAME),
)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.25},
)

In [6]:
contextualize_q_system_prompt = """
    Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}'),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
system_prompt = (
    """
    You are an internal technical support assistant for employees of a large company. 

    1. Use the entire conversation history with the user to understand the context of the inquiry.
    2. Incorporate the pieces of information retrieved from the knowledge base,
            which consists of pairs of questions (user inquiries) and fixed answers (responses).
    3. Based on the current conversation, choose the most relevant question
            from the retrieved pairs, and respond with the corresponding fixed answer from the knowledge base.
    4. You must provide responses exactly as they appear in the knowledge base,
            without any modifications. All responses must be in Russian.
    """
#     """
#     5.If no relevant answer is found, respond with the template:
#             'Чат-бот находится в стадии пилотирования и обучается ежедневно. Пожалуйста, обратитесь в поддержку или попробуйте переформулировать свой запрос.'
#     """
    '\n\n{context}'
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}'),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer',
)

In [9]:
response = conversational_rag_chain.invoke(
    {'input': 'Как взять отпуск?'},
    config={'configurable': {'session_id': '1'}},
)
response['answer']

'Создать заявку на отпуск можно в разделе "Сервис", подразделе "Отпуск". Требования к запрашиваемому отпуску вы сможете видеть левой части экрана.'